In [1]:
import os
import re
import pandas as pd
import io
import base64
from openai import OpenAI
from collections import defaultdict
import pdfplumber # 이미지 추출을 위해 유지
import filetype
import camelot # 테이블 추출을 위해 추가
from unstructured.partition.pdf import partition_pdf

In [10]:
# --- 1. AI 클라이언트 및 유틸리티 함수 정의 (기존과 동일) ---
client = OpenAI() # API 키는 환경 변수에 설정 권장

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def generate_image_description(image_path):
    print(f"  - 이미지 설명 생성 요청: {os.path.basename(image_path)}")
    base64_image = encode_image_to_base64(image_path)
    image_ext = os.path.splitext(image_path)[1].lstrip('.')
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user",
                   "content": [{"type": "text", 
                                "text": "이 이미지는 무엇에 관한 것인가요? RAG 시스템에서 검색될 것을 가정하고, 이미지의 핵심 내용을 한글로 설명해주세요."}, 
                               {"type": "image_url", "image_url": {"url": f"data:image/{image_ext};base64,{base64_image}"}}]}],
        max_tokens=200
    )
    description = response.choices[0].message.content.strip()
    print(f"  - 설명 생성 완료: {description[:30]}...")
    return description

def generate_table_summary(table_csv_str):
    if len(table_csv_str) > 4000:
        table_csv_str = table_csv_str[:4000]
    print("  - 테이블 요약 생성 요청...")
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system",
                   "content": "당신은 데이터 분석가입니다. 주어진 표 데이터를 보고, 표의 핵심 내용을 한글 문장으로 요약하는 역할을 맡았습니다."},
                  {"role": "user", "content": f"다음은 CSV 형식의 표 데이터입니다. 이 표가 어떤 정보를 담고 있으며, 주요 특징이 무엇인지 요약해주세요.\n\n--- 데이터 시작 ---\n{table_csv_str}\n--- 데이터 끝 ---"}],
        max_tokens=300
    )
    summary = response.choices[0].message.content.strip()
    print(f"  - 요약 생성 완료: {summary[:30]}...")
    return summary

# --- 2. AI 기능이 통합된 처리 함수 (camelot 데이터 형식에 맞게 수정) ---

def process_table_with_ai_camelot(table_df, output_dir, page_num, table_index):
    """camelot으로 추출한 테이블 DataFrame을 처리하고 AI 요약을 생성합니다."""
    try:
        df = table_df
        table_filename = f"p{page_num}_tbl{table_index}.csv"
        table_path = os.path.join(output_dir, "tables", table_filename)
        df.to_csv(table_path, index=False, encoding="utf-8-sig")
        
        # AI 요약을 위해 DataFrame을 CSV 문자열로 변환
        csv_string = df.to_csv(index=False)
        description = generate_table_summary(csv_string)
        
        return {"path": table_path, "page": page_num, "dataframe": df, "description": description}
    except Exception as e:
        print(f"  - 테이블 처리 중 오류 발생: {e}")
        return None

# process_image_with_ai 함수는 기존과 동일하게 pdfplumber를 사용하므로 변경 없음
def process_image_with_ai(image_data, output_dir, page_num, image_index):
    try:
        image_bytes = image_data['stream'].get_data()
        kind = filetype.guess(image_bytes)
        image_ext = kind.extension if kind else "png"
        image_filename = f"p{page_num}_img{image_index}.{image_ext}"
        image_path = os.path.join(output_dir, "images", image_filename)
        with open(image_path, "wb") as img_file:
            img_file.write(image_bytes)
        description = generate_image_description(image_path)
        return {"path": image_path, "page": page_num, "description": description}
    except Exception as e:
        print(f"  - 이미지 처리 중 오류 발생: {e}")
        return None

# --- 3. camelot을 사용하도록 수정한 메인 함수 ---

def create_integrated_markdown_from_camelot(pdf_path):
    output_dir = os.path.basename(pdf_path).split(".")[0]
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "tables"), exist_ok=True)

    # 1단계: pdfplumber로 이미지, camelot으로 테이블 미리 추출
    print("1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...")
    
    # pdfplumber로 이미지 추출
    images_by_page = defaultdict(list)
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            images_by_page[page.page_number].extend(page.images)
    print(f"총 {sum(len(v) for v in images_by_page.values())}개의 이미지 정보를 찾았습니다.")

    # camelot으로 테이블 추출
    # flavor='lattice'는 선이 있는 표에 적합, 선이 없는 표는 'stream' 사용
    tables_by_page = defaultdict(list)
    try:
        tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice', suppress_stdout=True)
        for table in tables:
            tables_by_page[table.page].append(table.df)
        print(f"총 {sum(len(v) for v in tables_by_page.values())}개의 테이블 정보를 찾았습니다.")
    except Exception as e:
        print(f"Camelot으로 테이블 추출 중 오류 발생: {e}")

    # 2단계: unstructured로 PDF 요소 순서대로 파티셔닝 (기존과 동일)
    print("2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...")
    elements = partition_pdf(filename=pdf_path, infer_table_structure=True, strategy="hi_res")

    # 3단계: 요소 순회하며 최종 마크다운 및 데이터 생성
    print("3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...")
    final_markdown_parts = []
    extracted_tables = []
    extracted_images = []
    page_counters = defaultdict(lambda: {'tables': 0, 'images': 0})

    for el in elements:
        page_num = el.metadata.page_number
        
        if "unstructured.documents.elements.Table" in str(type(el)):
            table_index = page_counters[page_num]['tables']
            if table_index < len(tables_by_page[page_num]):
                # camelot이 추출한 DataFrame을 처리
                raw_table_df = tables_by_page[page_num][table_index]
                table_data = process_table_with_ai_camelot(raw_table_df, output_dir, page_num, table_index)
                if table_data:
                    placeholder = f"\n\n[[-- TABLE: Page {page_num}, Index {table_index} | Path: {table_data['path']} --]]\n**표 요약:** {table_data['description']}\n\n"
                    final_markdown_parts.append(placeholder)
                    extracted_tables.append(table_data)
                    page_counters[page_num]['tables'] += 1
            
        elif "unstructured.documents.elements.Image" in str(type(el)):
            image_index = page_counters[page_num]['images']
            if image_index < len(images_by_page[page_num]):
                # pdfplumber가 추출한 이미지 처리
                raw_image_data = images_by_page[page_num][image_index]
                image_data = process_image_with_ai(raw_image_data, output_dir, page_num, image_index)
                if image_data:
                    relative_path = os.path.relpath(image_data['path'], output_dir).replace(os.sep, '/')
                    md_link = f"\n\n![{image_data['description']}]({relative_path})\n\n"
                    final_markdown_parts.append(md_link)
                    extracted_images.append(image_data)
                    page_counters[page_num]['images'] += 1
        
        else:
            final_markdown_parts.append(el.text)

    # 4단계: 최종 결과 정리 및 저장 (기존과 동일)
    print("4단계: 최종 결과 정리 및 저장...")
    final_markdown = "\n\n".join(final_markdown_parts)
    output_filename = os.path.join(output_dir, "integrated_markdown_camelot.md")
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(final_markdown)
    
    print("\n--- 작업 완료 ---")
    return {
        "output_dir": output_dir,
        "integrated_markdown_file": output_filename,
        "images": extracted_images,
        "tables": extracted_tables,
        "integrated_markdown_content": final_markdown,
    }

# --- 코드 실행 예제 ---
pdf_path = "datasets/manual.pdf" # 실제 PDF 파일 경로로 변경해주세요.
extracted_data = create_integrated_markdown_from_camelot(pdf_path)

# # 결과 확인
# print(f"\nAI 요약/설명이 포함된 통합 마크다운 파일이 '{extracted_data['integrated_markdown_file']}'에 저장되었습니다.")

CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


1단계: 미디어 정보 미리 추출 중 (이미지: pdfplumber, 테이블: camelot)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 0개의 이미지 정보를 찾았습니다.


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


총 2개의 테이블 정보를 찾았습니다.
2단계: unstructured로 문서 구조 분석 중 (hi_res 전략 사용)...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


3단계: 텍스트, 테이블, 이미지를 통합하고 AI로 내용을 보강합니다...
  - 테이블 요약 생성 요청...
  - 요약 생성 완료: 이 표는 특정 기간에 따른 신청 시기와 지급 비율에 대...
  - 테이블 요약 생성 요청...
  - 요약 생성 완료: 주어진 표는 연구 분야의 인건비 지급 기준과 자격 기준...
4단계: 최종 결과 정리 및 저장...

--- 작업 완료 ---


In [11]:
from pprint import pprint

pprint(extracted_data)

{'images': [],
 'integrated_markdown_content': 'Ⅴ 사업비 집행 및 정산지침\n'
                                '\n'
                                'OE\n'
                                '\n'
                                '사업비 지급 및 집행기간\n'
                                '\n'
                                '□ 사업비 지급\n'
                                '\n'
                                '\n'
                                '\n'
                                '[[-- TABLE: Page 1, Index 0 | Path: '
                                'manual\\tables\\p1_tbl0.csv --]]\n'
                                '**표 요약:** 이 표는 특정 기간에 따른 신청 시기와 지급 비율에 대한 정보를 '
                                "담고 있습니다. 각 분기별로 지급 비율이 다르게 나타나며, '협약 체결 시'에는 "
                                "20%, '협약 체결 후 2개월 이내'에서는 46%의 지급 비율이 있는 반면, "
                                "'중간 평가 후 1개월 이내'는 9%, '4분기(10월)'는 25%의 지급 비율을 "
                                '나타냅니다. 주요 특징으로는 신청 시기에 따른 지급 비율의 차이를 시각적으로 '
                                '고찰할 수

In [ ]:
import torch
import re
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document  # Document 클래스를 import 합니다.

# --- 1. 메타데이터를 포함하여 청킹하고 Document 객체를 생성하는 함수 ---

def create_documents_from_markdown(markdown_content, source_path):
    """
    논리적 청킹을 수행하고, 각 청크를 메타데이터가 포함된 LangChain Document 객체로 변환합니다.

    Args:
        markdown_content (str): 청킹할 마크다운 전체 텍스트
        source_path (str): 원본 파일 경로 (메타데이터에 사용)

    Returns:
        list[Document]: 내용과 메타데이터가 포함된 Document 객체 리스트
    """
    # 기존의 논리적 청킹 함수를 그대로 사용합니다.
    text_chunks = chunk_markdown_logically(markdown_content)
    
    documents = []
    for chunk_text in text_chunks:
        metadata = {"source": source_path}  # 모든 청크에 기본 출처 정보 추가
        
        # 청크 내용을 기반으로 추가 메타데이터 파싱
        if chunk_text.startswith("[[-- TABLE:"):
            metadata['type'] = 'table'
            # 정규표현식을 사용하여 페이지 번호와 경로 추출
            page_match = re.search(r"Page (\d+)", chunk_text)
            path_match = re.search(r"Path: (.*?)\s*--]]", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if path_match:
                metadata['table_path'] = path_match.group(1).strip()

        elif chunk_text.startswith("!["):
            metadata['type'] = 'image'
            # 정규표현식을 사용하여 페이지 번호와 이미지 설명 추출
            page_match = re.search(r"Image from page (\d+)", chunk_text)
            desc_match = re.search(r"!\[(.*?)\]\(", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if desc_match:
                metadata['description'] = desc_match.group(1).strip()
        else:
            metadata['type'] = 'text'

        # Document 객체 생성
        doc = Document(page_content=chunk_text, metadata=metadata)
        documents.append(doc)
        
    return documents

# --- 2. 기존 로직 실행 및 all_chunks 생성 ---

# chunk_markdown_logically 함수는 제공된 코드에 이미 정의되어 있다고 가정합니다.
def chunk_markdown_logically(markdown_content):
    """
    문서의 구조(제목, 표, 이미지)를 이해하여 논리적인 단위로 청킹합니다.
    RAG 시스템에 가장 효과적인 방법입니다.
    """
    # 1. 기본 블록으로 분리
    blocks = markdown_content.split('\n\n')
    
    chunks = []
    current_chunk = ""

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        # 2. 규칙에 따라 청크 생성
        is_table_placeholder = block.startswith("[[-- TABLE:")
        is_image_link = block.startswith("![")
        # 제목으로 사용될 수 있는 패턴들 (필요에 따라 추가)
        is_heading = block.startswith(('□', '○', '※')) or re.match(r'^[0-9]+\s|^\w+\s', block) and len(block) < 50

        if is_table_placeholder or is_image_link:
            # 테이블/이미지는 독립적인 청크로 처리
            if current_chunk:
                chunks.append(current_chunk.strip())
            chunks.append(block)
            current_chunk = ""
        elif is_heading:
            # 제목은 다음 블록과 합치기 위해, 진행 중인 청크를 먼저 저장
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = block
        else:
            # 일반 텍스트는 현재 청크(제목 또는 이전 텍스트)에 추가
            if current_chunk:
                current_chunk += "\n\n" + block
            else:
                current_chunk = block
    
    # 마지막 남은 청크 추가
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

source_file_path = "manual/integrated_markdown_camelot.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 'all_chunks' 변수는 이제 단순 문자열 리스트가 아닌,
# 내용과 메타데이터를 모두 포함한 Document 객체의 리스트가 됩니다.
all_chunks = create_documents_from_markdown(content, source_file_path)


# --- 생성된 Document 객체 확인 (상위 2개) ---
print("--- 생성된 Document 객체 (상위 2개) ---")
for i, doc in enumerate(all_chunks[:2]):
    print(f"--- Chunk {i+1} ---")
    print(f"Page Content:\n{doc.page_content[:200]}...\n") # 내용이 길 수 있으므로 일부만 출력
    print(f"Metadata:\n{doc.metadata}\n")


--- 생성된 Document 객체 (상위 2개) ---
--- Chunk 1 ---
Page Content:
Ⅴ 사업비 집행 및 정산지침

OE...

Metadata:
{'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}

--- Chunk 2 ---
Page Content:
사업비 지급 및 집행기간...

Metadata:
{'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}


--- Vector Store 생성 시작 ---
임베딩 모델을 위한 디바이스 설정: cuda

--- Vector Store 생성 및 저장 완료 ---
'./chroma_db' 디렉토리에 38개의 청크가 저장되었습니다.


In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

source_file_path = "manual/integrated_markdown_camelot.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    file_content = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # 청크 크기를 1000으로 설정
    chunk_overlap=150,  # 중첩 크기를 150으로 설정
    length_function=len, # 텍스트 길이를 계산할 함수 지정
)

# .split_text() 메서드를 사용하여 텍스트를 분할합니다.
split_texts = text_splitter.split_text(file_content)

In [57]:
# --- 3. Vector Store 생성 및 저장 ---

print("\n--- Vector Store 생성 시작 ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"임베딩 모델을 위한 디바이스 설정: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

# Chroma.from_documents 함수는 Document 객체 리스트를 받도록 설계되어 있습니다.
vectorstore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # DB를 저장할 디렉토리
)

print("\n--- Vector Store 생성 및 저장 완료 ---")
print(f"'{vectorstore._persist_directory}' 디렉토리에 {len(all_chunks)}개의 청크가 저장되었습니다.")


--- Vector Store 생성 시작 ---
임베딩 모델을 위한 디바이스 설정: cuda

--- Vector Store 생성 및 저장 완료 ---
'./chroma_db' 디렉토리에 38개의 청크가 저장되었습니다.


In [58]:
pprint(all_chunks)

[Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='Ⅴ 사업비 집행 및 정산지침\n\nOE'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='사업비 지급 및 집행기간'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='□ 사업비 지급'),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'table', 'page': 1, 'table_path': 'manual\\tables\\p1_tbl0.csv'}, page_content="[[-- TABLE: Page 1, Index 0 | Path: manual\\tables\\p1_tbl0.csv --]]\n**표 요약:** 이 표는 특정 기간에 따른 신청 시기와 지급 비율에 대한 정보를 담고 있습니다. 각 분기별로 지급 비율이 다르게 나타나며, '협약 체결 시'에는 20%, '협약 체결 후 2개월 이내'에서는 46%의 지급 비율이 있는 반면, '중간 평가 후 1개월 이내'는 9%, '4분기(10월)'는 25%의 지급 비율을 나타냅니다. 주요 특징으로는 신청 시기에 따른 지급 비율의 차이를 시각적으로 고찰할 수 있으며, 각각의 시기가 지급에 미치는 영향을 분석할 수 있는 자료임을 알 수 있습니다."),
 Document(metadata={'source': 'manual/integrated_markdown_camelot.md', 'type': 'text'}, page_content='※ 지급비율은 변동될 수 있으며, 중간평가 결과에

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

retriever = vectorstore.as_retriever()


prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)


# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
llm = OllamaLLM(model="gemma3:4b")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [64]:
question = "2회차 사업비 지급 비율은?"
answer = chain.invoke(question)
print(answer)

관련 정보를 찾을 수 없습니다.


<class 'langchain_core.documents.base.Document'>


In [ ]:
import fitz  # PyMuPDF
import pdfplumber
import os
import pandas as pd
import markdownify
import base64
from openai import OpenAI
import json

client = OpenAI()

def extract_pdf_elements(pdf_path):
    """
    PDF 파일에서 텍스트, 이미지, 표를 추출하여 별도로 저장하는 함수

    Args:
        pdf_path (str): 처리할 PDF 파일의 경로

    Returns:
        dict: 추출된 텍스트, 이미지 정보, 표 정보가 담긴 딕셔너리
    """
    
    output_dir = os.path.basename(pdf_path).split(".")[0]
    image_dir = os.path.join(output_dir, "images")
    table_dir = os.path.join(output_dir, "tables")
    
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(table_dir, exist_ok=True)
    
    all_text = []
    image_info = []
    table_info = []

    doc = fitz.open(pdf_path)
    
    with pdfplumber.open(pdf_path) as plumber_pdf:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            plumber_page = plumber_pdf.pages[page_num]
            
            # 텍스트 추출
            all_text.append(page.get_text("html"))
            
            
            # 이미지 추출
            images = page.get_images(full=True)
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                
                image_filename = f"p{page_num + 1}_img{img_index}.{image_ext}"
                image_path = os.path.join(image_dir, image_filename)
                
                with open(image_path, "wb") as img_file:
                    img_file.write(image_bytes)
                    
                    image_info.append({
                        "path":image_path,
                        "page":page_num+1,
                        "description":""
                    })
                    
            # 테이블 추출
            tables = plumber_page.extract_tables()
            for table_index, table in enumerate(tables):
                df = pd.DataFrame(table[1:], columns=table[0])
                
                table_filename = f"p{page_num + 1}_tbl{table_index}.csv"
                table_path = os.path.join(table_dir, table_filename)
                
                df.to_csv(table_path, index=False, encoding="utf-8-sig")
                
                table_info.append({
                    "path":table_path,
                    "page":page_num+1,
                    "description":"",
                    "dataframe":df
                })
                
    doc.close()
    
    full_text = "\n".join(all_text)
    full_markdown = markdownify.markdownify(full_text, heading_style="ATX")
    
    text_filename = os.path.join(output_dir, "full_markdown.txt")
    with open(text_filename, "w", encoding="utf-8") as f:
        f.write(full_markdown)
        
    return {
        "output_dir":output_dir,
        "text_file":text_filename,
        "images":image_info,
        "tables":table_info,
        "text_per_page": full_markdown
    }
    
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def generate_image_description(image_path):
    base64_image = encode_image_to_base64(image_path)
    image_ext = os.path.splitext(image_path)[1].lstrip('.')
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "이 이미지는 무엇에 관한 것인가요? RAG 시스템에서 검색될 것을 가정하고, 이미지의 핵심 내용을 간결하게 한글로 설명해주세요."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/{image_ext};base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=200
    )
    return response.choices[0].message.content

def generate_table_summary(table_csv_str):
    if len(table_csv_str) > 4000:
        table_csv_str = table_csv_str[:4000]
        
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "당신은 데이터 분석가입니다. 주어진 표 데이터를 보고, 표의 핵심 내용을 간결하게 한글 문장으로 요약하는 역할을 맡았습니다."
            },
            {
                "role": "user",
                "content": f"다음은 표 데이터입니다. 이 표가 어떤 정보를 담고 있으며, 주요 특징이 무엇인지 한두 문장으로 요약해주세요.\n\n--- 데이터 시작 ---\n{table_csv_str}\n--- 데이터 끝 ---"
            }
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

In [ ]:
pdf_path = "datasets/여비산정기준표.pdf"
extracted_data = extract_pdf_elements(pdf_path)

for i, image_item in enumerate(extracted_data["images"]):
    description = generate_image_description(image_item['path'])
    image_item["description"] = description
    print(description)
    
for i, table_item in enumerate(extracted_data["tables"]):
    description = generate_table_summary(table_item['path'])
    table_item["description"] = description
    print(description)
    

output_dir = extracted_data['output_dir']
json_data_path = os.path.join(output_dir, 'enriched_data.json')
# DataFrame 객체는 JSON으로 바로 저장되지 않으므로 제외
for tbl in extracted_data['tables']:
    del tbl['dataframe']
    
with open(json_data_path, 'w', encoding='utf-8') as f:
    json.dump(extracted_data, f, ensure_ascii=False, indent=4)

죄송합니다. 이 이미지는 회색 배경이라서 구체적인 내용을 알 수 없습니다. 
이 표는 여비 산정에 대한 기준을 담고 있으며, 출장 및 여행 시 필요한 비용 산정의 근거와 기준을 명확히 제시하는 정보를 포함하고 있습니다.
주어진 데이터는 '여비산정기준표'라는 제목을 가진 표로, 여비 산정을 위한 기준이나 규정을 담고 있는 것으로 보입니다. 구체적인 항목이나 내용은 주어지지 않았지만, 여행 경비나 출장을 위한 비용 산정에 관련된 정보를 제공하는 것 같습니다.
표 데이터는 여행 경비를 산정하는 기준을 담고 있는 것으로 보입니다. 주요 특징은 출장이나 여행 시 필요한 경비를 효율적으로 계산할 수 있도록 가이드라인을 제공하는 것입니다.
제공된 표는 여비 산정을 위한 기준을 담고 있으며, 출장 시 교통비, 숙박비 등의 경비를 산정하는 표준을 제시하고 있습니다. 주요 특징은 각 항목별로 세부 산정 기준이 명시되어 있다는 점입니다.
표 데이터는 여비 산정에 관한 기준을 제시하고 있는 것으로 보입니다. 주로 출장 등에 필요한 여비를 계산하는 데 필요한 기준과 요율이 포함되어 있을 가능성이 높습니다.


In [ ]:
content_map = {
    i + 1: [page_text] for i, page_text in enumerate(extracted_data["text_per_page"])
}

with open(json_data_path, 'r', encoding='utf-8') as f:
    enriched_data = json.load(f)
    
for image in enriched_data["images"]:
    page_num = image["page"]
    description = image["description"]
    
    formatted_desc = f"\n[이미지 파일: {os.path.basename(image['path'])}]\n이미지 설명: {description}\n"
    if page_num in content_map:
        content_map[page_num].append(formatted_desc)
        
for table in enriched_data["tables"]:
    page_num = table["page"]
    summary = table["description"]
    
    formatted_summary = f"\n[표 파일: {os.path.basename(table['path'])}]\n표 요약: {summary}\n"
    if page_num in content_map:
        content_map[page_num].append(formatted_summary)

In [ ]:
final_pages_content = []
for page_num in sorted(content_map.keys()):
    page_content = '\n'.join(content_map[page_num])
    final_pages_content.append(page_content)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 마크다운에 적합한 스플리터
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    # 마크다운 구조를 우선적으로 고려하여 분리
    separators=["\n## ", "\n### ", "\n#### ", "\n\n", "\n", " ", ""],
    length_function=len
)

all_chunks = []
for i, page_text in enumerate(final_pages_content):
    page_num = i+1
    chunks = text_splitter.create_documents(
        texts=[page_text],
        metadatas=[{"source_page":page_num}]
    )
    all_chunks.extend(chunks)

In [ ]:
import torch
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document  # Document 클래스를 import 합니다.

# --- 1. 메타데이터를 포함하여 청킹하고 Document 객체를 생성하는 함수 ---

def create_documents_from_markdown(markdown_content, source_path):
    """
    논리적 청킹을 수행하고, 각 청크를 메타데이터가 포함된 LangChain Document 객체로 변환합니다.

    Args:
        markdown_content (str): 청킹할 마크다운 전체 텍스트
        source_path (str): 원본 파일 경로 (메타데이터에 사용)

    Returns:
        list[Document]: 내용과 메타데이터가 포함된 Document 객체 리스트
    """
    # 기존의 논리적 청킹 함수를 그대로 사용합니다.
    text_chunks = chunk_markdown_logically(markdown_content)
    
    documents = []
    for chunk_text in text_chunks:
        metadata = {"source": source_path}  # 모든 청크에 기본 출처 정보 추가
        
        # 청크 내용을 기반으로 추가 메타데이터 파싱
        if chunk_text.startswith("[[-- TABLE:"):
            metadata['type'] = 'table'
            # 정규표현식을 사용하여 페이지 번호와 경로 추출
            page_match = re.search(r"Page (\d+)", chunk_text)
            path_match = re.search(r"Path: (.*?)\s*--]]", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if path_match:
                metadata['table_path'] = path_match.group(1).strip()

        elif chunk_text.startswith("!["):
            metadata['type'] = 'image'
            # 정규표현식을 사용하여 페이지 번호와 이미지 설명 추출
            page_match = re.search(r"Image from page (\d+)", chunk_text)
            desc_match = re.search(r"!\[(.*?)\]\(", chunk_text)
            if page_match:
                metadata['page'] = int(page_match.group(1))
            if desc_match:
                metadata['description'] = desc_match.group(1).strip()
        else:
            metadata['type'] = 'text'

        # Document 객체 생성
        doc = Document(page_content=chunk_text, metadata=metadata)
        documents.append(doc)
        
    return documents

# --- 2. 기존 로직 실행 및 all_chunks 생성 ---

# chunk_markdown_logically 함수는 제공된 코드에 이미 정의되어 있다고 가정합니다.
# def chunk_markdown_logically(markdown_content): ...

source_file_path = "manual/integrated_markdown_plumber.md"

with open(source_file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 'all_chunks' 변수는 이제 단순 문자열 리스트가 아닌,
# 내용과 메타데이터를 모두 포함한 Document 객체의 리스트가 됩니다.
all_chunks = create_documents_from_markdown(content, source_file_path)


# --- 생성된 Document 객체 확인 (상위 2개) ---
print("--- 생성된 Document 객체 (상위 2개) ---")
for i, doc in enumerate(all_chunks[:2]):
    print(f"--- Chunk {i+1} ---")
    print(f"Page Content:\n{doc.page_content[:200]}...\n") # 내용이 길 수 있으므로 일부만 출력
    print(f"Metadata:\n{doc.metadata}\n")


# --- 3. Vector Store 생성 및 저장 ---

print("\n--- Vector Store 생성 시작 ---")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"임베딩 모델을 위한 디바이스 설정: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

# Chroma.from_documents 함수는 Document 객체 리스트를 받도록 설계되어 있습니다.
vectorstore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # DB를 저장할 디렉토리
)

print("\n--- Vector Store 생성 및 저장 완료 ---")
print(f"'{vectorstore._persist_directory}' 디렉토리에 {len(all_chunks)}개의 청크가 저장되었습니다.")

--- 생성된 Document 객체 (상위 2개) ---
--- Chunk 1 ---
Page Content:
Ⅴ 사업비 집행 및 정산지침

rau...

Metadata:
{'source': 'manual/integrated_markdown_plumber.md', 'type': 'text'}

--- Chunk 2 ---
Page Content:
사업비 지급 및 집행기간...

Metadata:
{'source': 'manual/integrated_markdown_plumber.md', 'type': 'text'}


--- Vector Store 생성 시작 ---
임베딩 모델을 위한 디바이스 설정: mps


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, 
    QFileDialog, QLineEdit, QTextEdit, QFrame
)
from PyQt6.QtCore import Qt

class DragDropLabel(QLabel):
    """드래그 앤 드롭을 지원하는 커스텀 라벨 위젯"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.setText("여기에 PDF 파일을 드래그 앤 드롭 하거나\n아래 버튼을 클릭하세요.")
        self.setStyleSheet("""
            QLabel {
                border: 2px dashed #aaa;
                padding: 20px;
                font-size: 14px;
                background-color: #f9f9f9;
            }
        """)

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            if file_path.lower().endswith('.pdf'):
                print(f"드롭된 파일: {file_path}")
                self.parent().upload_file(file_path)
                return
        print("PDF 파일이 아닙니다.")

class RAGClientApp(QWidget):
    def __init__(self):
        super().__init__()
        # Flask 서버 주소
        self.upload_url = "http://127.0.0.1:5000/upload"
        self.ask_url = "http://127.0.0.1:5000/ask"
        self.init_ui()

    def init_ui(self):
        """애플리케이션의 전체 UI를 설정합니다."""
        main_layout = QVBoxLayout()

        # --- 1. 파일 업로드 섹션 ---
        upload_group_label = QLabel("1. PDF 파일 업로드")
        upload_group_label.setStyleSheet("font-size: 16px; font-weight: bold; margin-top: 10px;")
        main_layout.addWidget(upload_group_label)
        
        self.drop_label = DragDropLabel(self)
        main_layout.addWidget(self.drop_label)

        self.btn_select = QPushButton("컴퓨터에서 PDF 파일 선택")
        self.btn_select.clicked.connect(self.open_file_dialog)
        main_layout.addWidget(self.btn_select)
        
        self.status_label = QLabel("상태: 대기 중")
        self.status_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        main_layout.addWidget(self.status_label)
        
        # --- 구분선 ---
        separator = QFrame()
        separator.setFrameShape(QFrame.Shape.HLine)
        separator.setFrameShadow(QFrame.Shadow.Sunken)
        main_layout.addWidget(separator)

        # --- 2. 질문/답변 섹션 ---
        qa_group_label = QLabel("2. 질문하기")
        qa_group_label.setStyleSheet("font-size: 16px; font-weight: bold; margin-top: 10px;")
        main_layout.addWidget(qa_group_label)

        # 질문 입력 레이아웃
        question_layout = QHBoxLayout()
        self.question_input = QLineEdit()
        self.question_input.setPlaceholderText("업로드된 PDF에 대해 질문을 입력하세요...")
        question_layout.addWidget(self.question_input)

        self.btn_ask = QPushButton("질문하기")
        self.btn_ask.clicked.connect(self.ask_question)
        question_layout.addWidget(self.btn_ask)
        main_layout.addLayout(question_layout)

        # 답변 출력창
        answer_label = QLabel("답변:")
        main_layout.addWidget(answer_label)
        
        self.answer_output = QTextEdit()
        self.answer_output.setReadOnly(True) # 답변은 수정할 수 없도록 설정
        self.answer_output.setStyleSheet("background-color: #f0f0f0;")
        main_layout.addWidget(self.answer_output)
        
        # 초기에는 질문 섹션을 비활성화
        self.set_qa_section_enabled(False)

        self.setLayout(main_layout)
        self.setWindowTitle("PDF 기반 RAG 질문 답변 시스템")
        self.setGeometry(300, 300, 500, 600)

    def set_qa_section_enabled(self, enabled):
        """질문/답변 섹션의 활성화 상태를 설정합니다."""
        self.question_input.setEnabled(enabled)
        self.btn_ask.setEnabled(enabled)
        self.answer_output.setEnabled(enabled)
        if not enabled:
            self.answer_output.setPlaceholderText("PDF 파일을 먼저 업로드해주세요.")
        else:
            self.answer_output.setPlaceholderText("")


    def open_file_dialog(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "PDF 파일 선택", "", "PDF Files (*.pdf)")
        if file_path:
            self.upload_file(file_path)

    def upload_file(self, file_path):
        filename = file_path.split('/')[-1]
        self.status_label.setText(f"상태: '{filename}' 업로드 및 처리 중... (파일 크기에 따라 시간이 걸릴 수 있습니다)")
        self.set_qa_section_enabled(False) # 처리 중 다시 비활성화
        QApplication.processEvents() # UI가 멈추지 않도록 이벤트 처리

        try:
            with open(file_path, 'rb') as f:
                files = {'file': (filename, f, 'application/pdf')}
                # RAG 처리는 오래 걸릴 수 있으므로 timeout을 넉넉하게 설정
                response = requests.post(self.upload_url, files=files, timeout=300)
            
            response.raise_for_status()
            
            response_data = response.json()
            self.status_label.setText(f"상태: {response_data.get('message', '업로드 및 처리 완료!')}")
            self.set_qa_section_enabled(True) # 처리가 끝나면 질문 섹션 활성화

        except requests.exceptions.RequestException as e:
            self.status_label.setText(f"상태: 업로드 실패 - {e}")
        except Exception as e:
            self.status_label.setText(f"상태: 오류 발생 - {e}")

    def ask_question(self):
        question = self.question_input.text().strip()
        if not question:
            self.answer_output.setText("질문을 입력해주세요.")
            return

        self.answer_output.setText("답변을 생성 중입니다...")
        QApplication.processEvents()

        try:
            payload = {"question": question}
            response = requests.post(self.ask_url, json=payload, timeout=60)
            response.raise_for_status()

            response_data = response.json()
            self.answer_output.setText(response_data.get("answer", "답변을 가져올 수 없습니다."))

        except requests.exceptions.RequestException as e:
            self.answer_output.setText(f"오류: 서버와 통신할 수 없습니다.\n{e}")
        except Exception as e:
            self.answer_output.setText(f"알 수 없는 오류가 발생했습니다.\n{e}")


if __name__ == '__main__':
    try:
        import requests
    except ImportError:
        print("requests 라이브러리가 필요합니다. 'pip install -r requirements.txt' 명령어로 설치해주세요.")
        sys.exit(1)
        
    app = QApplication(sys.argv)
    ex = RAGClientApp()
    ex.show()
    sys.exit(app.exec())



2025-06-14 22:03:20.273 python[64743:7789898] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


SystemExit: 0

/opt/miniconda3/envs/rag/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
